# Анализ базы данных сервиса для чтения книг

**Цель проекта –** проанализировать базу данных сервиса для чтения книг по подписке.

**В качестве входных данных используется база данных, содержащая:**

- Информацию о книгах; 
- Издательствах; 
- Авторах; 
- Пользовательские обзоры книг.

**Необходимо:**

1. Сделать обзор данных;
2. Посчитать, сколько книг вышло после 1 января 2000 года;
3. Для каждой книги посчитать количество обзоров и среднюю оценку;
4. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, чтобы исключить из анализа брошюры;
5. Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
6. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
pd.set_option("display.precision", 2)

In [2]:
# устанавливаем параметры
db_config = {'user': '', # имя пользователя
             'pwd': '',  # пароль
             'host': '', # хост
             'port': '', # порт подключения
             'db': ''}   # название базы данных

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'],
                                                         db_config['db'])

In [4]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Обзор данных

In [5]:
def sql_request(query):
    """
    Функция запроса к базе данных и возврата ответа в виде pandas датафрейма

    query - запрос
    """

    return pd.io.sql.read_sql(query, con = engine)

In [6]:
def get_info(table):
    """
    Функция для вывода информации о таблице

    table - название таблицы
    """

    query = f"""
    SELECT column_name,data_type 
    FROM information_schema.columns 
    WHERE
    table_name = '{table}' 
    """
    
    return sql_request(query)

In [7]:
def get_head(table):
    """
    Функция для вывода первых 5 строк из таблицы

    table - название таблицы
    """

    query = f"""
    SELECT *
    FROM {table}
    LIMIT 5 
    """
    
    return sql_request(query)

### В таблице books

In [8]:
get_info('books')

,column_name,data_type
0,book_id,integer
1,author_id,integer
2,title,text
3,num_pages,integer
4,publication_date,date
5,publisher_id,integer


In [9]:
get_head('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Согласно документации к данным:**

books - содержит данные о книгах.

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц; 
- `publication_date` — дата публикации книги; 
- `publisher_id` — идентификатор издателя.

### В таблице authors

In [10]:
get_info('authors')

,column_name,data_type
0,author_id,integer
1,author,text


In [11]:
get_head('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Согласно документации к данным:**

authors - содержит данные об авторах.

- `author_id `— идентификатор автора; 
- `author` — имя автора.

### В таблице publishers

In [12]:
get_info('publishers')

,column_name,data_type
0,publisher_id,integer
1,publisher,text


In [13]:
get_head('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Согласно документации к данным:**

publishers - содержит данные об издательствах.

- `publisher_id `— идентификатор издательства; 
- `publisher` — название издательства.

### В таблице ratings

In [14]:
get_info('ratings')

,column_name,data_type
0,rating_id,integer
1,book_id,integer
2,username,text
3,rating,integer


In [15]:
get_head('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Согласно документации к данным:**

ratings - содержит данные о пользовательских оценках книг.

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку; 
- `rating` — оценка книги.

### В таблице reviews

In [16]:
get_info('reviews')

,column_name,data_type
0,review_id,integer
1,book_id,integer
2,username,text
3,text,text


In [17]:
get_head('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Согласно документации к данным:**

reviews - содержит данные о пользовательских обзорах на книги.

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор; 
- `text` — текст обзора.

## Количество книг вышедших после 1 января 2000 года

In [18]:
query = """
SELECT COUNT(*)
FROM books
WHERE publication_date > '2000-01-01'
"""

In [19]:
sql_request(query)

,count
0,819


*После 1 января 2000 года вышло 819 книг.*

## Количество обзоров и средняя оценка для каждой книги

In [20]:
query = """
WITH review_cnt AS (
SELECT book_id,
       COUNT(review_id) AS review_cnt
FROM reviews
GROUP BY book_id
),
rating_avg AS (
SELECT book_id, 
       AVG(rating) AS rating_avg
FROM ratings
GROUP BY book_id
)
SELECT b.title AS title,
       rc.review_cnt,
       ra.rating_avg
FROM books as b
LEFT JOIN review_cnt AS rc ON b.book_id = rc.book_id
LEFT JOIN rating_avg AS ra ON b.book_id = ra.book_id
WHERE review_cnt is not null
ORDER BY review_cnt DESC, rating_avg DESC
"""

In [21]:
sql_request(query)

,title,review_cnt,rating_avg
0,Twilight (Twilight #1),7,3.66
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,The Book Thief,6,4.26
4,The Glass Castle,6,4.21
...,...,...,...
989,Moo Baa La La La!,1,3.00
990,Debt of Honor (Jack Ryan #7),1,3.00
991,History of Beauty,1,2.50
992,Tsubasa: RESERVoir CHRoNiCLE Vol. 1,1,2.50


*Больше всего обзоров на книгу "Twilight (Twilight #1)" ее средний ретинг 3.66, есть книги, на которые еще не написали обзоры.*

## Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [22]:
query = """
SELECT p.publisher AS publisher,
       COUNT(b.book_id) AS book_cnt
FROM publishers as p
JOIN books AS b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY publisher
ORDER BY book_cnt DESC
LIMIT 1
"""

In [23]:
sql_request(query)

,publisher,book_cnt
0,Penguin Books,42


*Наибольшее число книг выпустило издательство "Penguin Books", их количество составило 42 и он же ответ на самый главный вселенский вопрос.*

## Автор с самой высокой средней оценкой книг, с количеством оценок более 50

In [24]:
query = """
WITH rating_by_books AS(
    SELECT book_id,
           COUNT(rating_id) AS rating_cnt
    FROM ratings
    GROUP BY book_id
)
SELECT a.author,
       AVG(rt.rating) AS rating_avg
FROM authors as a
JOIN books AS b ON a.author_id = b.author_id
JOIN rating_by_books AS r ON b.book_id = r.book_id
JOIN ratings AS rt on b.book_id = rt.book_id
WHERE r.rating_cnt >= 50
GROUP BY a.author
ORDER BY rating_avg DESC
LIMIT 1
"""

In [25]:
sql_request(query)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.29


*Автором с самой высокой средней оценкой среди книг с количеством оценок более 50 оказалась "J.K. Rowling/Mary GrandPré".*

## Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [26]:
query = """
WITH rating_over_50 AS(
    SELECT username,
           COUNT(rating_id) AS rating_cnt
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
),
review_by_users AS(
    SELECT username,
           COUNT(review_id) AS review_cnt
    FROM reviews 
    GROUP BY username
)
SELECT AVG(rbu.review_cnt)
FROM rating_over_50 AS ro
JOIN review_by_users AS rbu ON ro.username = rbu.username
"""

In [27]:
sql_request(query)

,avg
0,24.33


*Среднее количество обзоров 24.33*

## Общий вывод

- База данных представляет из себя 5 таблиц:
  - `books` - содержит данные о книгах;
  - `authors` - содержит данные об авторах;
  - `publishers` - содержит данные об издательствах;
  - `rating` - содержит данные о пользовательских обзорах;
  - `reviews` - содержит данные о пользовательских обзорах$
- В ходе анализа были получены ответы на следующие вопросы:
  - Количество книг вышедших после 1 января 2000 года – `819`;
  - Количество обзоров и средняя оценка для каждой книги:
    1. Место – `Twilight`, количество обзоров `7`, средний рейтинг `3.66`;
    2. Место – `Harry Potter and the Prisoner of Azkaban`, количество обзоров	`6`, средний рейтинг	`4.41`;
    3. Место – `Harry Potter and the Chamber of Secrets`, количество обзоров	`6`, средний рейтинг	`4.29`;
  - Издательство, которое выпустило наибольшее число книг толще 50 страниц – `Penguin Books`;
  - Автор с самой высокой средней оценкой книг, с количеством оценок более 50 – `J.K. Rowling/Mary GrandPré`;
  - Среднее количество обзоров от пользователей, которые поставили больше 50 оценок – `24.33`. 